<a href="https://colab.research.google.com/github/gkv856/KaggleData/blob/main/NLP_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import random
import math
!pip install bert-for-tf2
!pip install tensorflow_text
import tensorflow_text as text 

     |████████████████████████████████| 41 kB 147 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=014a184863b8e94342d0318cb3d05e54a869e2851efd21815c3ba5c4f7669456
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=d875a163f04267edbade614fa503d3e9643a09db919f6ed7a2ca275cec843c9c
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7912 sha256=bb52f82ab7fb1b770ba40a815c0ca1be21a5b6892f06f8e3aaf56b9e2aa0d369
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params
     |████████████████████████████████| 4.4 MB 5.3 MB/s 


In [2]:
TRAIN_URL = "https://raw.githubusercontent.com/gkv856/KaggleData/main/train.csv"
TEST_URL = "https://raw.githubusercontent.com/gkv856/KaggleData/main/test.csv"
SUBMISSION_URL = "https://raw.githubusercontent.com/gkv856/KaggleData/main/sample_submission.csv"


In [3]:
df_train_original = pd.read_csv(TRAIN_URL)


In [6]:
mispell_dict = {"aren't" : "are not",
                "ain't": "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"}

def replace_typical_misspell(text):
    mispellings_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))

    def replace(match):
        return mispell_dict[match.group(0)]

    return mispellings_re.sub(replace, text)


def clean_text(text_str):
  text_str = BeautifulSoup(text_str, "lxml").get_text()
  text_str = re.sub(r"@[A-Za-z0-9]+", " ", text_str)
  text_str = re.sub(r"https?://[[A-Za-z0-9./]+", " ", text_str)
  text_str = re.sub(r"[^a-zA-Z.!?']", " ", text_str)
  text_str = re.sub(r" +", " ", text_str)
  text_str = text_str.lower()
  # text_str = replace_typical_misspell(text_str)

  return text_str

In [7]:
df_train = df_train_original.copy()
df_train["text"] = df_train["text"].apply(clean_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: FutureWarning: Possible nested set at position 10


In [8]:
df_train["text_len"] = df_train["text"].map(len)

In [9]:
from sklearn.model_selection import train_test_split

data = df_train["text"].values
labels = df_train["target"].values
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=.3, random_state=42)

In [10]:
import bert

# will use this tokenizer to convert sentence to tokens
FullTokenizer = bert.bert_tokenization.FullTokenizer

BERT_PP_MODEL_URL = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
# BERT_MODEL_URL = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
BERT_MODEL_URL = "https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/4"
# preprocessor = hub.load(BERT_PP_MODEL_URL)
# tokenize = hub.KerasLayer(preprocessor.tokenize)

In [11]:
X_train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# y_train_ds = tf.data.Dataset.from_tensor_slices(y_train)

X_test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
# y_test_ds = tf.data.Dataset.from_tensor_slices(y_test)

# X_train_ds, y_train_ds, X_test_ds, y_test_ds

In [12]:
BATCH_SIZE = 32
# train_dataset = train_dataset.batch(BATCH_SIZE).map(prepare_data, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
X_train_ds_bp = X_train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
# y_train_ds_bp = y_train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

X_test_ds_bp = X_test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
# y_test_ds_bp = y_test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# X_train_ds_bp, y_train_ds_bp, X_test_ds_bp, y_test_ds_bp

In [13]:
import tensorflow.keras.layers as layers

In [14]:
bert_preprocessor = hub.KerasLayer(BERT_PP_MODEL_URL)
bert_encoder_NT = hub.KerasLayer(BERT_MODEL_URL, trainable=False)
# bert_encoder_T = hub.KerasLayer(BERT_MODEL_URL, trainable=True)

In [15]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import *

--2021-09-16 17:43:34--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-09-16 17:43:34 (84.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [16]:
lr = 0.001
input_layer = layers.Input(shape=(), dtype=tf.string, name="batched_text_inputs_layer")
encoder_inputs = bert_preprocessor(input_layer)
bert_outputs = bert_encoder_NT(encoder_inputs)
sequence_output = bert_outputs["sequence_output"]


dense1 = layers.Dense(128, activation="relu")(sequence_output)
dropout1 = layers.Dropout(0.3)(dense1)

dense2 = layers.Dense(8, activation="relu")(dropout1)


# pool1 = layers.GlobalAveragePooling1D()(dense2)
pool1 = layers.GlobalMaxPool1D()(dense2)

# pool1 = layers.Flatten()(dense2)

outputs = layers.Dense(1, activation="sigmoid", name="output")(pool1)

model2 = tf.keras.Model(input_layer, outputs, name="model2")

model2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
               metrics=["accuracy"])

model2.summary()

Model: "model2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
batched_text_inputs_layer (Inpu [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_mask': (None 0           batched_text_inputs_layer[0][0]  
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'encoder_outputs':  335141889   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
_____________________________________________________________________________________________

In [ ]:
history_model2 = model2.fit(X_train_ds_bp,
                            validation_data=X_test_ds_bp,
                            epochs=5)
plot_loss_curves(history_model2)

Epoch 1/5
167/167 [==============================] - 430s 2s/step - loss: 0.4650 - accuracy: 0.7896 - val_loss: 0.4198 - val_accuracy: 0.8244
Epoch 2/5
167/167 [==============================] - 407s 2s/step - loss: 0.4053 - accuracy: 0.8219 - val_loss: 0.4096 - val_accuracy: 0.8271
Epoch 3/5
167/167 [==============================] - 407s 2s/step - loss: 0.3752 - accuracy: 0.8388 - val_loss: 0.4029 - val_accuracy: 0.8284
